# 🚀 Massenscraping von Pressemitteilungen

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b>
</br>
Die Lernenden können mit Hilfe eines Jupyter Notebooks Python-Code zur Extraktion des Website-Texts ausführen.</br>
</div>

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht

Im Folgenden werden alle Pressemitteilungen der Berliner Staatskanzlei gescraped

Dafür werden folgendene Schritte durchgeführt:
1. Wir werden die Struktur des Teils der Website untersuchen, der alle Pressemitteilungen enthält.
2. Wir werden die URL-Links zu allen Pressemitteilungen abrufen.
3. Abschließend werden wir alle Pressemitteilungen scrapen.

In [ ]:
# 🚀 Install libraries 
!pip install requests tqdm 

In [ ]:
import requests, pathlib, time, re, logging, textwrap
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

## Abruf und Analyse der Suchseite für Pressemitteilungen

Im Kapitel ['Aufbau des Forschungskorpus'](../corpus_collection/corpus-collection_building-our-corpus.html#aufbau-des-forschungskorpus) haben wir die Auswahl- und Filterprozesse für unser Korpus von Pressemitteilungen beschrieben. Nun geht es darum, das Korpus mithilfe von Scraping-Tools und HTML-Kenntnissen zu extrahieren. <!-- In the chapter ['Aufbau des Forschungskorpus'](../corpus_collection/corpus-collection_building-our-corpus.html#aufbau-des-forschungskorpus) we have outlined the selection & filtering process for our corpus of press releases. Now it is time to imlement scraping of tha corpus using scraping tools and knowledge of HTML. -->

1. Wir wissen bereits, dass das [Suchmenü](https://www.berlin.de/presse/pressemitteilungen/index/search) auf der Website Berlin.de gezielt die Auswahl der für uns interessanten Abteilungen ermöglicht. <!-- We already know that the [Search menu](https://www.berlin.de/presse/pressemitteilungen/index/search) on the Berlin.de website allows to select only the departments that interest us: -->
   
![selection](../book_images/selection_of_depts.png)

2. Anschließend können wir mit den [ausgewählten Abteilungen und einer leeren Suchanfrage suchen](https://www.berlin.de/presse/pressemitteilungen/index/search/?searchtext=&boolean=0&startdate=&enddate=&alle-senatsverwaltungen=on&institutions%5B%5D=Presse-+und+Informationsamt+des+Landes+Berlin&institutions%5B%5D=Senatsverwaltung+für+Bildung%2C+Jugend+und+Familie&institutions%5B%5D=Senatsverwaltung+für+Finanzen&institutions%5B%5D=Senatsverwaltung+für+Inneres+und+Sport&institutions%5B%5D=Senatsverwaltung+für+Arbeit%2C+Soziales%2C+Gleichstellung%2C+Integration%2C+Vielfalt+und+Antidiskriminierung&institutions%5B%5D=Senatsverwaltung+für+Justiz+und+Verbraucherschutz&institutions%5B%5D=Senatsverwaltung+für+Kultur+und+Gesellschaftlichen+Zusammenhalt&institutions%5B%5D=Senatsverwaltung+für+Stadtentwicklung%2C+Bauen+und+Wohnen&institutions%5B%5D=Senatsverwaltung+für+Mobilität%2C+Verkehr%2C+Klimaschutz+und+Umwelt&institutions%5B%5D=Senatsverwaltung+für+Wirtschaft%2C+Energie+und+Betriebe&institutions%5B%5D=Senatsverwaltung+für+Wissenschaft%2C+Gesundheit+und+Pflege&alle-bezirksamt=on&institutions%5B%5D=Bezirksamt+Charlottenburg-Wilmersdorf&institutions%5B%5D=Bezirksamt+Friedrichshain-Kreuzberg&institutions%5B%5D=Bezirksamt+Lichtenberg&institutions%5B%5D=Bezirksamt+Marzahn-Hellersdorf&institutions%5B%5D=Bezirksamt+Mitte&institutions%5B%5D=Bezirksamt+Neukölln&institutions%5B%5D=Bezirksamt+Pankow&institutions%5B%5D=Bezirksamt+Reinickendorf&institutions%5B%5D=Bezirksamt+Spandau&institutions%5B%5D=Bezirksamt+Steglitz-Zehlendorf&institutions%5B%5D=Bezirksamt+Tempelhof-Schöneberg&institutions%5B%5D=Bezirksamt+Treptow-Köpenick&alle-landesbeauftragte=on&institutions%5B%5D=Beauftragte+des+Senats+für+Integration+und+Migration&institutions%5B%5D=Beauftragter+zur+Aufarbeitung+der+SED-Diktatur&institutions%5B%5D=Bürger-+und+Polizeibeauftragter+des+Landes+Berlin&institutions%5B%5D=Pflegebeauftragte+des+Landes+Berlin&institutions%5B%5D=Landestierschutzbeauftragte&institutions%5B%5D=Landeswahlleitung&bt=) und so alle Pressemitteilungen dieser Abteilungen abrufen: <!-- After that we can [perform search with selected depatrmentes and an empty query](https://www.berlin.de/presse/pressemitteilungen/index/search/?searchtext=&boolean=0&startdate=&enddate=&alle-senatsverwaltungen=on&institutions%5B%5D=Presse-+und+Informationsamt+des+Landes+Berlin&institutions%5B%5D=Senatsverwaltung+für+Bildung%2C+Jugend+und+Familie&institutions%5B%5D=Senatsverwaltung+für+Finanzen&institutions%5B%5D=Senatsverwaltung+für+Inneres+und+Sport&institutions%5B%5D=Senatsverwaltung+für+Arbeit%2C+Soziales%2C+Gleichstellung%2C+Integration%2C+Vielfalt+und+Antidiskriminierung&institutions%5B%5D=Senatsverwaltung+für+Justiz+und+Verbraucherschutz&institutions%5B%5D=Senatsverwaltung+für+Kultur+und+Gesellschaftlichen+Zusammenhalt&institutions%5B%5D=Senatsverwaltung+für+Stadtentwicklung%2C+Bauen+und+Wohnen&institutions%5B%5D=Senatsverwaltung+für+Mobilität%2C+Verkehr%2C+Klimaschutz+und+Umwelt&institutions%5B%5D=Senatsverwaltung+für+Wirtschaft%2C+Energie+und+Betriebe&institutions%5B%5D=Senatsverwaltung+für+Wissenschaft%2C+Gesundheit+und+Pflege&alle-bezirksamt=on&institutions%5B%5D=Bezirksamt+Charlottenburg-Wilmersdorf&institutions%5B%5D=Bezirksamt+Friedrichshain-Kreuzberg&institutions%5B%5D=Bezirksamt+Lichtenberg&institutions%5B%5D=Bezirksamt+Marzahn-Hellersdorf&institutions%5B%5D=Bezirksamt+Mitte&institutions%5B%5D=Bezirksamt+Neukölln&institutions%5B%5D=Bezirksamt+Pankow&institutions%5B%5D=Bezirksamt+Reinickendorf&institutions%5B%5D=Bezirksamt+Spandau&institutions%5B%5D=Bezirksamt+Steglitz-Zehlendorf&institutions%5B%5D=Bezirksamt+Tempelhof-Schöneberg&institutions%5B%5D=Bezirksamt+Treptow-Köpenick&alle-landesbeauftragte=on&institutions%5B%5D=Beauftragte+des+Senats+für+Integration+und+Migration&institutions%5B%5D=Beauftragter+zur+Aufarbeitung+der+SED-Diktatur&institutions%5B%5D=Bürger-+und+Polizeibeauftragter+des+Landes+Berlin&institutions%5B%5D=Pflegebeauftragte+des+Landes+Berlin&institutions%5B%5D=Landestierschutzbeauftragte&institutions%5B%5D=Landeswahlleitung&bt=) and retrieve all press releases belonging to these departments:-->

![suchergebnisse](../book_images/suchergebnisse.png)

Wir sehen, dass die Links hier in einer Tabelle gespeichert sind. In HTML wird eine Tabelle mit dem `<table>`-Element dargestellt. Wenn wir den Quellcode dieser Seite betrachten, stellen wir fest, dass sie eine Tabelle enthält, in der alle Links aufgeführt sind:

![selection](../book_images/pm_table_source_html.png) 

Um diese Links zu durchsuchen, können wir die grundlegenden HTML-Abfragefunktionen der bereits bekannten Bibliothek BeautifulSoup verwenden. Das machen wir im nächsten Abschnitt.

## Suchergebnisse scrapen und Pressemitteilungen extrahieren (auf einer Seite): 

In [ ]:
# -- organise data ----------------------------------------------------
FIRST_OUTPUT_PAGE = (
    "https://www.berlin.de/presse/pressemitteilungen/index/search/?searchtext=&boolean=0&startdate=&enddate=&alle-senatsverwaltungen=on&institutions%5B%5D=Presse-+und+Informationsamt+des+Landes+Berlin&institutions%5B%5D=Senatsverwaltung+für+Bildung%2C+Jugend+und+Familie&institutions%5B%5D=Senatsverwaltung+für+Finanzen&institutions%5B%5D=Senatsverwaltung+für+Inneres+und+Sport&institutions%5B%5D=Senatsverwaltung+für+Arbeit%2C+Soziales%2C+Gleichstellung%2C+Integration%2C+Vielfalt+und+Antidiskriminierung&institutions%5B%5D=Senatsverwaltung+für+Justiz+und+Verbraucherschutz&institutions%5B%5D=Senatsverwaltung+für+Kultur+und+Gesellschaftlichen+Zusammenhalt&institutions%5B%5D=Senatsverwaltung+für+Stadtentwicklung%2C+Bauen+und+Wohnen&institutions%5B%5D=Senatsverwaltung+für+Mobilität%2C+Verkehr%2C+Klimaschutz+und+Umwelt&institutions%5B%5D=Senatsverwaltung+für+Wirtschaft%2C+Energie+und+Betriebe&institutions%5B%5D=Senatsverwaltung+für+Wissenschaft%2C+Gesundheit+und+Pflege&alle-bezirksamt=on&institutions%5B%5D=Bezirksamt+Charlottenburg-Wilmersdorf&institutions%5B%5D=Bezirksamt+Friedrichshain-Kreuzberg&institutions%5B%5D=Bezirksamt+Lichtenberg&institutions%5B%5D=Bezirksamt+Marzahn-Hellersdorf&institutions%5B%5D=Bezirksamt+Mitte&institutions%5B%5D=Bezirksamt+Neukölln&institutions%5B%5D=Bezirksamt+Pankow&institutions%5B%5D=Bezirksamt+Reinickendorf&institutions%5B%5D=Bezirksamt+Spandau&institutions%5B%5D=Bezirksamt+Steglitz-Zehlendorf&institutions%5B%5D=Bezirksamt+Tempelhof-Schöneberg&institutions%5B%5D=Bezirksamt+Treptow-Köpenick&alle-landesbeauftragte=on&institutions%5B%5D=Beauftragte+des+Senats+für+Integration+und+Migration&institutions%5B%5D=Beauftragter+zur+Aufarbeitung+der+SED-Diktatur&institutions%5B%5D=Bürger-+und+Polizeibeauftragter+des+Landes+Berlin&institutions%5B%5D=Pflegebeauftragte+des+Landes+Berlin&institutions%5B%5D=Landestierschutzbeauftragte&institutions%5B%5D=Landeswahlleitung&bt="
) 
DATA_DIR   = pathlib.Path("../data")         
HTML_DIR   = DATA_DIR / "html"
TXT_DIR    = DATA_DIR / "txt"
HTML_DIR.mkdir(parents=True, exist_ok=True)
TXT_DIR.mkdir(parents=True,  exist_ok=True)
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

In [ ]:
# -- helper -----------------------------------------------------------------

def get_soup(url: str) -> BeautifulSoup:
    """Download a page and return BeautifulSoup (retry politely on transient errors)."""
    while True:
        r = requests.get(
            url,
            timeout=20,
            headers={"User-Agent": "Mozilla/5.0 (compatible; QuadrigaScraper/1.0)"}
        )
        if r.status_code == 200:
            return BeautifulSoup(r.text, "lxml")
        logging.warning("Status %s on %s – retrying in 5 s", r.status_code, url)
        time.sleep(5)

def slugify(text_: str, maxlen: int = 60) -> str:
    """Rough filename-safe slug for headlines."""
    text_ = re.sub(r"\W+", "-", text_.lower()).strip("-")
    return text_[:maxlen] or "untitled"

<!-- <button onclick="toggleDisplay('myDIV1')">Was passiert hier oben? (Schritt-für-Schritt-Erklärung)</button>

<div id="myDIV1" style="display:none"> -->
    
### Was passiert in diesem Codeblock oben?

1. **`get_soup()`**

   * Die Funktion versucht, eine Webseite herunterzuladen und sofort als `BeautifulSoup`-Objekt zurückzugeben.
   * Sie sendet einen HTTP-Request mit

     * 20 Sekunden Timeout (schützt vor ewig hängenden Verbindungen) und
     * einem eigenen *User-Agent*-Header, damit der Server weiß, dass es sich um ein automatisiertes, aber höfliches Skript handelt („QuadrigaScraper/1.0“).
   * Wenn der Server **Status 200** liefert → der HTML-Text wird geparst und zurückgegeben.
   * Bei jedem anderen Statuscode wird eine Warnung ins Log geschrieben, 5 Sekunden gewartet und dann erneut versucht. Dadurch bricht der Scraper nicht sofort ab, sondern behandelt temporäre Fehler (z. B. 500er oder Netzwerk-Glitches) selbstständig.

2. **`slugify()`**

   * Wandelt eine Überschrift (oder beliebigen Text) in einen dateisystemtauglichen „Slug“ um.
   * Schritte:

     1. alles in Kleinbuchstaben verwandeln,
     2. alle Zeichen, die **nicht** Buchstaben, Ziffern oder Unterstrich sind, durch Bindestriche ersetzen (`\W+`),
     3. führende/abschließende Bindestriche entfernen,
     4. das Ergebnis auf maximal 60 Zeichen kürzen (damit Dateinamen handlich bleiben).
   * Falls nach allen Filterungen nichts übrig ist, liefert die Funktion sicherheitshalber den Platzhalter **„untitled“** zurück.

<!-- </div>

<script>

function toggleDisplay(id) {
  var x = document.getElementById(id);
  if (x.style.display === "none") {
    x.style.display = "block";
  } else {
    x.style.display = "none";
  }
}
</script> -->

In [ ]:
# -- step 1: parse ONE results page -----------------------------------------
search_soup = get_soup(FIRST_OUTPUT_PAGE)

rows = search_soup.select("table tbody tr")  
records = []
print(f"Found {len(rows)} rows on the page.")

for tr in tqdm(rows, desc="Rows"):
    # grab all data cells once
    cells = tr.find_all("td")
    if len(cells) < 3:          # footer / empty rows → ignore
        continue

    # column 1 – date
    date_txt = cells[0].get_text(strip=True)

    # column 2 – headline + link
    anchor = cells[1].find("a", href=True)
    if anchor is None:          # safety check
        continue
    title   = anchor.get_text(strip=True)
    pr_url  = "https://www.berlin.de" + anchor["href"]

    # column 3 – issuing authority (“Ressort”)
    ressort = cells[2].get_text(strip=True)

    # deterministic ID, e.g. 1570469
    uid = anchor["href"].split("/")[-1].split(".")[-2]

        # -- step 2: download the press release itself -----------
    html_file = HTML_DIR / f"{uid}.html"
    txt_file  = TXT_DIR  / f"{uid}.txt"

    if not html_file.exists():        # skip if already scraped
        pr_soup = get_soup(pr_url)

        # write raw HTML
        html_file.write_text(str(pr_soup), encoding="utf-8")

        # extract main text; fallback to whole page if CSS id changes
        body = (pr_soup.select_one("#article") or        # new layout (2024)
                pr_soup.select_one("#content") or        # classic layout
                pr_soup)                                 # last resort
        clean_text = body.get_text(" ", strip=True)
        txt_file.write_text(clean_text, encoding="utf-8")
    else:
        # we still need the plain text length for the DataFrame below
        clean_text = txt_file.read_text(encoding="utf-8")

    records.append(
        dict(
            date=date_txt,
            ressort=ressort,
            title=title,
            pr_url=pr_url,
            filename_html=html_file.name,
            filename_txt=txt_file.name,
            n_tokens=len(clean_text.split())
        )
    )
    time.sleep(0.4)      # politeness

# -- step 3: inspect the harvested metadata ---------------------------------
df = pd.DataFrame(records)
df.head()          # normal Jupyter display is fine ― no extra libraries

<!-- <button onclick="toggleDisplay('myDIV2')">Was passiert hier oben? (Schritt-für-Schritt-Erklärung)</button>

<div id="myDIV2" style="display:none"> -->

### Was passiert in diesem Codeblock oben? – Schritt für Schritt

1. **Erste Seite einlesen**

   ```python
   search_soup = get_soup(SAMPLE_OUTPUT_PAGE)
   rows = search_soup.select("table tbody tr")
   ```

   * Die Funktion `get_soup()` lädt genau **eine** Ergebnisseite der
     Such-/Listenansicht und gibt sie als Beautiful-Soup-Objekt zurück.
   * Mit dem CSS-Selektor `table tbody tr` werden alle Tabellenzeilen
     der Ergebnisliste eingesammelt. Jede Zeile repräsentiert eine
     einzelne Pressemitteilung.

2. **Vorbereitung für die Schleife**

   ```python
   records = []
   print(f"Found {len(rows)} rows on the page.")
   ```

   * `records` soll später eine Liste von Dictionaries für das
     DataFrame sammeln.
   * Eine kurze Ausgabe zeigt, wie viele Zeilen tatsächlich gefunden
     wurden – nützlich für Kontroll-/Debug-Zwecke.

3. **Iterieren mit Fortschrittsbalken**

   ```python
   for tr in tqdm(rows, desc="Rows"):
   ```

   * `tqdm` liefert einen hübschen Fortschrittsbalken – perfekt für
     Lehr- und Live-Demos.

4. **Zellen extrahieren & Plausibilität prüfen**

   ```python
   cells = tr.find_all("td")
   if len(cells) < 3:          # footer / empty rows → ignore
       continue
   ```

   * Alle `<td>` einer Zeile werden auf einmal geholt.
   * Hat eine Zeile weniger als drei Zellen, handelt es sich um
     Paginierungs- oder Leerzeilen; die werden übersprungen.

5. **Spalte 1 – Datum**

   ```python
   date_txt = cells[0].get_text(strip=True)
   ```

   * `strip=True` entfernt Zeilenumbrüche und Leerzeichen – wir erhalten
     saubere Strings wie „16.06.2025“.

6. **Spalte 2 – Überschrift & Link**

   ```python
   anchor = cells[1].find("a", href=True)
   if anchor is None:
       continue
   title  = anchor.get_text(strip=True)
   pr_url = "https://www.berlin.de" + anchor["href"]
   ```

   * Innerhalb der zweiten Zelle steckt der anklickbare Link.
   * Sicherheits-Check: Falls doch kein `<a>` vorhanden ist, Zeile
     überspringen.
   * Die relative URL wird zur vollständigen URL ergänzt.

7. **Spalte 3 – Ressort (herausgebende Behörde)**

   ```python
   ressort = cells[2].get_text(strip=True)
   ```

8. **Eindeutige ID ableiten**

   ```python
   uid = anchor["href"].split("/")[-1].split(".")[-2]
   ```

   * Vom Pfadsegment `pressemitteilung.1570469.php` wird mittels
     `split()` das numerische Stück **1570469** herausgelöst.
   * Diese ID landet später im Dateinamen, damit jeder Release genau
     eine HTML- und eine TXT-Datei bekommt.

9. **Dateipfade festlegen**

   ```python
   html_file = HTML_DIR / f"{uid}.html"
   txt_file  = TXT_DIR  / f"{uid}.txt"
   ```

10. **HTML herunterladen & Text extrahieren (nur falls neu)**

    ```python
    if not html_file.exists():
        pr_soup = get_soup(pr_url)
        html_file.write_text(str(pr_soup), encoding="utf-8")

        body = (pr_soup.select_one("#article")     # neues Layout
                or pr_soup.select_one("#content")  # altes Layout
                or pr_soup)                        # Fallback
        clean_text = body.get_text(" ", strip=True)
        txt_file.write_text(clean_text, encoding="utf-8")
    else:
        clean_text = txt_file.read_text(encoding="utf-8")
    ```

    * **Idempotenz**: Wenn die Datei schon existiert, wird nichts
      erneut heruntergeladen – das spart Zeit und Traffic.
    * Der eigentliche Text sitzt mal in `#article`, mal in
      `#content`. Wir probieren beide Selektoren und greifen im Zweifel
      auf die ganze Seite zurück.
    * HTML und gereinigter Plain-Text werden getrennt gespeichert.

11. **Metadaten sammeln**

    ```python
    records.append(
        dict(
            date=date_txt,
            ressort=ressort,
            title=title,
            pr_url=pr_url,
            filename_html=html_file.name,
            filename_txt=txt_file.name,
            n_tokens=len(clean_text.split())
        )
    )
    ```

    * Alle wesentlichen Infos – inklusive Dateinamen und Token-Anzahl –
      landen in einem Dictionary, das wir später direkt in ein
      DataFrame gießen.

12. **Höfliche Pause**

    ```python
    time.sleep(0.4)
    ```

    * 400 ms warten verringert die Gefahr, den Server zu überlasten.

13. **Auswertung in Pandas**

    ```python
    df = pd.DataFrame(records)
    df.head()
    ```

    * Am Ende verwandeln wir die gesammelten Dictionaries in ein
      `DataFrame`, um die ersten Zeilen gleich im Notebook
      inspizieren zu können.

So wird auf anschauliche Weise demonstriert, wie man **gezielt Teile einer
HTML-Tabelle parst**, die Detailseiten herunterlädt, Text extrahiert und alles
sauber für weitere Analysen ablegt.

<!-- </div> -->


## Massenscraping von Pressemitteilungen

In [ ]:
# ╔═╡ 80_pagination_helpers
# Basis-URL ohne "/page/<nr>"-Segment,
# aber *inklusive* aller Query-Parameter (= Filter der langen Such-URL)

SEARCH_ROOT = FIRST_OUTPUT_PAGE

def last_page_number() -> int:
    """
    Ermittelt über das <nav>-Element ('pager-skip-to-last') die höchste
    Ergebnisseiten-Nummer.
    """
    soup = get_soup(SEARCH_ROOT)                            # 1. Suchseite laden
    last_link = soup.select_one("li.pager-skip-to-last a")
    if not last_link:
        raise RuntimeError("Konnte die letzte Seite nicht finden – Selector?")
    
    # href hat die Form ".../page/5239?<query>"
    m = re.search(r"/page/(\d+)", last_link["href"])
    if not m:
        raise RuntimeError("Seitenzahl nicht im href gefunden.")
    return int(m.group(1))

def page_url(page_num: int) -> str:
    """
    Baut die URL für eine beliebige Seite nach folgendem Muster auf:
    <root>/page/<nr>?<identische Query-Parameter>
    """
    if page_num < 1:
        raise ValueError("Seitennummern beginnen bei 1.")
    return SEARCH_ROOT.replace("/search/", f"/search/page/{page_num}/")


In [ ]:
# ╔═╡ 81_bulk_crawler
def crawl_all_pages(pages: int | None = None, sleep_s: float = 0.4) -> pd.DataFrame:
    """
    Läuft über alle Treffer-Seiten, sammelt und lädt jede Pressemitteilung.
    Gibt ein DataFrame mit sämtlichen Metadaten zurück.
    
    *pages*  – Anzahl der Seiten; None → automatisch ermitteln  
    *sleep_s*– Höflichkeits-Delay zwischen Requests
    """
    if pages is None:
        pages = last_page_number()
        print(f"→ Letzte Trefferseite lautet {pages}")

    all_records: list[dict] = []
    for p in tqdm(range(1, pages + 1), desc="Result pages"):
        soup = get_soup(page_url(p))
        rows = soup.select("table tbody tr")
        
        for tr in rows:
            cells = tr.find_all("td")
            if len(cells) < 3:
                continue  # Skip Leer-/Footer-Zeilen

            date_txt = cells[0].get_text(strip=True)
            anchor   = cells[1].find("a", href=True)
            if anchor is None:
                continue
            title    = anchor.get_text(strip=True)
            pr_url   = "https://www.berlin.de" + anchor["href"]
            ressort  = cells[2].get_text(strip=True)
            uid      = anchor["href"].split(".")[-2]  # z. B. 1570469
            html_fp  = HTML_DIR / f"{uid}.html"
            txt_fp   = TXT_DIR  / f"{uid}.txt"

            # ----- Detailseite nur laden, wenn noch nicht vorhanden -----------
            if not html_fp.exists():
                pr_soup = get_soup(pr_url)
                html_fp.write_text(str(pr_soup), encoding="utf-8")
                body = (pr_soup.select_one("#article") or
                        pr_soup.select_one("#content") or
                        pr_soup)
                clean = body.get_text(" ", strip=True)
                txt_fp.write_text(clean, encoding="utf-8")
            else:
                clean = txt_fp.read_text(encoding="utf-8")

            all_records.append(
                dict(
                    date=date_txt,
                    ressort=ressort,
                    title=title,
                    pr_url=pr_url,
                    filename_html=html_fp.name,
                    filename_txt=txt_fp.name,
                    n_tokens=len(clean.split())
                )
            )
            time.sleep(sleep_s)   # Höflichkeit: Delay pro Detail-Request

    return pd.DataFrame(all_records)

In [ ]:
# ╔═╡ 82_test_run_5_pages
df_test = crawl_all_pages(pages=5)
df_test.head()

In [ ]:
# ╔═╡ 83_run_bulk
# Achtung: Das kann > 1 Stunde dauern und tausende Dateien erzeugen!
df_all = crawl_all_pages(pages=None)   # None → auto-detect
df_all.head()

### Export von Metadaten

In [ ]:
metadata_path = DATA_DIR / "metadata_updated.csv"
df_all.to_csv()